# Phase 4 AstroPi team Atlantes

## Introduction

In this phase 4 of the project, we will have the data generated in phase 3 by our program executed on the Raspberry on board the ISS. The objective of our project is to make music with the data obtained, that is, to proceed with its [sonification](https://en.wikipedia.org/wiki/Sonification). Once the sound is obtained we will make a [stopmotion](https://en.wikipedia.org/wiki/Stop_motion) movie with the photos obtained by the Raspberry camera to which we will add the sonification. 

The project proposed in phase 1 aimed to ask to a source of historical meteorological data to obtain the climatic conditions on the ground at the time and the places through which the ISS passed during the execution of our program. We will look for such a data source and analyze whether it provides us with suitable data for our purpose.

To analyze the data we will use graphical representations of the different columns of data obtained. In this way we will identify those that may be more suitable for sonification. We will therefore learn to draw several of these columns on the same graph in order to compare them.

Once we have decided on the information that we are going to sonify, we will use Python code to produce a sound file from the data as a result of our project.

## Preparing environment

In order for this notebook to run correctly, some packages must be installed on the Google Colab sandbox. We will also install the data collected on ISS and some files needed.

In [ ]:
!apt-get install -y fluidsynth fluid-soundfont-gm timidity ffmpeg
!pip install meteostat geopandas pretty-midi MIDIUtil matplotlib jupyter
!wget https://niubit.net/media/uploads/images/atlantes/atlantes_files_for_colab.zip -O atlantes_files_for_colab.zip
!unzip -o -q atlantes_files_for_colab.zip

The following code cell finishes setting up the environment and defines some variables.

In [ ]:
%pylab inline
import os

# Adjusting chart dimensions    
plt.rcParams["figure.figsize"] = (10, 8)                  # Size of charts in inches
plt.rcParams["figure.dpi"] = 90                           # Resolution of charts in DPIs

# Defining some environment variables for data sources
path = os.path.dirname(os.path.realpath("__file__"))
path_atlantes = path + "/atlantes"

RESULTS_FILE = "atlantes.csv"                             # ESA CSV file
RESULTS_FILE2 = "atlantes_stations.csv"                   # New CSV file with meteostat stations data
RESULTS_FILE3 = "atlantes_means.csv"                      # New CSV file with pictures data
RESULTS_FILE4 = "atlantes_stations_plus_hourly.csv"       # New CSV file with hourly meteostat data

## Meteorological data source

The best source found has turned out to be [Meteostat](https://meteostat.net/en). It is free, offers historical data and also has a [Python library](https://dev.meteostat.net/python/) to obtain the data from code. We will start by analyzing the Meteostat website by consulting, for example, the [historical data that can be consulted about Zaragoza](https://meteostat.net/en/place/ES-SIWO).

Analyzing the [basic example code of the meteostat library](https://dev.meteostat.net/python/#example), we see that the latitude/longitude coordinates entered as parameters in the code are used first to locate the closest weather station, from which the historical data will then be obtained. In other words, the data will almost never come from the exact location of the ISS, but from the nearest station. This makes sense, but it makes us realize that we may have a problem, since most of the time the ISS flies over the sea, where few stations are to be expected. When it does on ground, every 15 seconds (which is the interval between the photos and the data collection) approximately 100km are traveled, so at least in populated areas it is hoped that we will be able to achieve sufficient precision.

It will be the first thing we are going to analyze, that is, to what extent the ISS flyby positions are covered by Meteostat with sufficient precision. We are going to go through the CSV data file of our program (atlantes.csv) to locate the Meteostat weather station closest to each point to see how often the station changes or repeats. Then we will represent these stations on a terrestrial map and compare the points with those of the ISS positions.

## Learning to work with data

Before, we have to learn a series of Python techniques to access and handle the data from CSV file.

The following code block uses the [`pandas`](https://pandas.pydata.org/pandas-docs/stable/index.html) Python library to read the CSV file. It loads the file on a data structure that greatly facilitates the manipulation of data arrays in the style of a spreadsheet. We are going to start by printing the data structure directly on screen to see what we have in hands. As it will contain a lot of data, we are going to see only the first two rows, which we achieve with the `head()` function:

In [ ]:
import os
import csv
import pandas

# Building file path
file = os.path.join(path_atlantes, RESULTS_FILE)

# Loading CSV file on memory
data = pandas.read_csv(file)
cut_data = data.head(2)                 # We get only the first two rows

print(cut_data)

When we want a single column of data, it will be as simple as indicating the name of the column (the first value of the column in the CSV file that acts as a header) separated from the data structure with a period: 

In [ ]:
import os
import csv
import pandas

file = os.path.join(path_atlantes, RESULTS_FILE)

data = pandas.read_csv(file)
sub_data = data.latitude                # Data subset with 'latitude' column
cut_data = sub_data.head(2)             # We get only the first two rows

print(cut_data)

When we want a set of columns, that is, a kind of subset of the CSV sheet, we will use the [`loc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) property as follows:

In [ ]:
import os
import csv
import pandas

file = os.path.join(path_atlantes, RESULTS_FILE)

data = pandas.read_csv(file)
# Data subset with 4 specific columns
sub_data = data.loc[:, ["datetime", "picture_file", "latitude", "longitude"]]
cut_data = sub_data.head(2)             # We get only the first two rows

print(cut_data)

Finally we will see how to iterate the data structure to process each row that it returns. To achieve this we will use the [`iterrows()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) function that returns an index and a dictionary with the data it contains for each row.

In [ ]:
import os
import csv
import pandas

file = os.path.join(path_atlantes, RESULTS_FILE)

data = pandas.read_csv(file)
# Data subset with 4 specific columns
sub_data = data.loc[:, ["datetime", "picture_file", "latitude", "longitude"]]
cut_data = sub_data.head(2)             # We get only the first two rows

for index, row in cut_data.iterrows():
    print(index)
    print(row["datetime"])
    print(row["picture_file"])
    print()

## Weather stations location

Now that we know how to read the data, we are going to start processing it. We'll start by preprocessing some data so we can graph it later. What we are going to do now is generate a new CSV file with some columns from the original file (that will serve as index to be able to cross the data between several files) to which we will add new columns with data from the weather stations closest to the latitude/longitude the ISS was in at each iteration.

The following code block takes the `atlantes.csv` file and does the following for each row:

1. Get latitde/longitude columns.
2. Get the weather station closest to that position from meteostat.
3. Record next data on a new CSV:
    * `datetime`: date/time from original file.
    * `picture_file`: picture filename from original file.
    * `latitude`: latitude from original file.
    * `longitude`: longitud from original file.
    * `station_id`: weather station ID on meteostat.
    * `station`: weather station name.
    * `country`: weather station country.
    * `st_lat`: weather station latitude.
    * `st_lon`: weather station longitude.

The result of all this will be the new file `atlantes_stations.csv` that we will generate in the same directory where the `atlantes.csv` data source was.

This time we are going to work with the complete set of data, so the execution of next code cell can take a long time to complete (about 5 minutes).

In [ ]:
import os
import csv
import pandas
from meteostat import Stations

file_in = os.path.join(path_atlantes, RESULTS_FILE)    # CSV input file complete path
file_out = os.path.join(path_atlantes, RESULTS_FILE2)  # CSV output file complete path
# Object of the meteostat library to consult weather stations of the meteostat database
stations = Stations()

file = open(file_out, 'w', newline='', encoding='utf-8')     # Opening output file in write mode (`w`)

# Reading specific columns from original file
data = pandas.read_csv(file_in).loc[:, ["datetime", "picture_file", "latitude", "longitude"]]

# Writing colum headers to output file
header = ("datetime", "picture_file", "latitude", "longitude", "station_id", "station", "country", "st_lat", "st_lon")
csv.writer(file).writerow(header)

# Iterating data in input file
for index, row in data.iterrows():
    # Searching for the closest weather station to the ISS position
    station = stations.nearby(row["latitude"], row["longitude"]).fetch(1)
    # Writing data to output file
    csv.writer(file).writerow((row["datetime"], row["picture_file"], row["latitude"], row["longitude"], station.index[0], station["name"][0], station["country"][0], station["latitude"][0], station["longitude"][0]))

# Closing output file
file.close()

# Logging end of process
print("Fichero '%s' generado." % (file_out))

So now, we have in the same file the position of the ISS in each iteration of our program, together with the position of the nearest weather station.

Now we are going to draw these positions on a plane (thanks to [this article](https://coderzcolumn.com/tutorials/data-science/plotting-static-maps-with-geopandas-working-with-geospatial-data) for the idea) in order to check if the data we will obtain from the stations will have sufficient precision.

The following code block draws on a Earth plane the positions of the ISS in blue and those of the nearest weather stations in red.

In [ ]:
import os
import pandas
import matplotlib.pyplot as plt
import geopandas

file = os.path.join(path_atlantes, RESULTS_FILE2)  # CSV input file complete path

data = pandas.read_csv(file)                       # Reading whole file

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

with plt.style.context(("seaborn", "ggplot")):
    world.plot(figsize=(18,10), color="white", edgecolor="grey")
    # Plotting weather stations closest to ISS (red)
    plt.scatter(data.st_lon, data.st_lat, zorder=1, alpha= 0.2, c='red', s=30)
    # Plotting ISS positions (blue)
    plt.scatter(data.longitude, data.latitude, zorder=1, alpha= 0.2, c='blue', s=30)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title("Meteostat stations closer to ISS path");

We can see how when the ISS flies over land areas (especially in America), there are numerous weather stations so that the two curves almost fit perfectly. However, in most areas, especially when it flies over the sea, the seasons make very big jumps. The points have been represented with a certain level of transparency, so that the level of opacity of the point indicates the number of times the same station is repeated. That is why the meteorological stations represented over the sea have a more intense red.

We can crop the graph to take a closer look at some areas, such as the american flyby, adjusting the ends of the X and Y axes.

In [ ]:
import os
import pandas
import matplotlib.pyplot as plt
import geopandas

file = os.path.join(path_atlantes, RESULTS_FILE2)     # CSV input file complete path

data = pandas.read_csv(file, nrows=419).tail(113)     # Reading American flyby data (307 -> 419)

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

with plt.style.context(("seaborn", "ggplot")):
    world.plot(figsize=(18,10), color="white", edgecolor="grey")
    # Plotting weather stations closest to ISS (red)
    plt.scatter(data.st_lon, data.st_lat, zorder=1, alpha= 0.2, c='red', s=30)
    # Plotting ISS positions (blue)
    plt.scatter(data.longitude, data.latitude, zorder=1, alpha= 0.2, c='blue', s=30)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title("Meteostat stations closer to ISS path");
    plt.gca().set_xlim(-135, -35)                          # Setting the X axis limits
    plt.gca().set_ylim(-30, 55)                            # Setting the Y axis limits

It is curious to see how in North America there is practically a weather station just below every position of ISS. This situation worsens as we enter South America.

Representing only the points of the stations helps to understand the problem.

In [ ]:
import os
import pandas
import matplotlib.pyplot as plt
import geopandas

file = os.path.join(path_atlantes, RESULTS_FILE2)  # CSV input file complete path

data = pandas.read_csv(file)                       # Reading whole file

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

with plt.style.context(("seaborn", "ggplot")):
    world.plot(figsize=(18,10), color="white", edgecolor="grey")
    # Plotting weather stations closest to ISS (red)
    plt.scatter(data.st_lon, data.st_lat, zorder=1, alpha= 0.2, c='red', s=30)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title("Meteostat stations closer to ISS path");

The next thing we will do is locate some interesting points in the data series. Visualizing the pictures and relying on the representation of the points that we have made on the earth plane, we can highlight the following points (we will indicate them with the name of the photo file):

|Picture|Hour|Latitude|Longitude|Zone|GMaps|Event|
|:---|:---|:------|:-------|:---|:----|:------|
|atlantes_038.jpg|19:55:44|-15.930611|-24.8557447|South Atlantic Ocean|[Link](https://goo.gl/maps/CHLZuX2VouyLZW1F9)|Sunset on Earth|
|atlantes_059.jpg|20:00:59|-30.835167|-11.039583|South Atlantic Ocean|[Link](https://goo.gl/maps/W1tcUhtfSUg75Pv27)|Sunset on ISS|
|atlantes_172.jpg|20:29:14|-22.955139|113.971111|Lyndon WA 6701, Australia|[Link](https://goo.gl/maps/k7pcNEWZQ8wMmfbG7)|We skim northwestern Australia (at night)|
|atlantes_186.jpg|20:32:44|-12.696889|122.401889|Indian Ocean|[Link](https://goo.gl/maps/NE26MTBpryoxowxN8)|Sunrise on ISS|
|atlantes_208.jpg|20:38:14|3.935500|134.405194|North Pacific Ocean|[Link](https://goo.gl/maps/Z3auiDeyTtsQGLXY9)|Sunrise on Earth|
|atlantes_307.jpg|21:03:00|50.315444|-127.637972|Mount Waddington B, BC, Canada|[Link](https://goo.gl/maps/TT29TYdrg9sQjVRp6)|Entering North America through Vancouver|
|atlantes_354.jpg|21:14:45|25.587417|-80.278167|Biscayne Bay, Florida, USA|[Link](https://goo.gl/maps/vTiZMuaKD4oBCAVZA)|Leaving North America through Florida|
|atlantes_361.jpg|21:16:30|20.581278|-75.737639|Parque Nacional la Mensura, Cuba|[Link](https://goo.gl/maps/TYtAw2YighVEzSyKA)|Crossing Cuba|
|atlantes_375.jpg|21:20:00|10.210778|-67.528250|Francisco Linares Alcántara, Aragua, Venezuela|[Link](https://goo.gl/maps/Jafv6Diq6jrubAxL8)|Entering South America through Venezuela|
|atlantes_409.jpg|21:28:30|-15.460444|-48.835306|Vila Propício - State of Goiás, Brazil|[Link](https://goo.gl/maps/XmUCCiBxFYbwRfz56)|Sunset on Earth|
|atlantes_419.jpg|21:31:00|-22.755972|-42.721722|Pitangas, Tanguá - State of Rio de Janeiro, Brazil|[Link](https://goo.gl/maps/XCbTjLmB2pYCdT8z8)|Leaving South America through Brazil (at night)|
|atlantes_431.jpg|21:34:00|-31.078889|-34.374528|South Atlantic Ocean|[Link](https://goo.gl/maps/TaNCgeFUYze6ocJS9)|Sunset on ISS|
|atlantes_558.jpg|22:05:45|-12.424111|98.999056|Indian Ocean|[Link](https://goo.gl/maps/PbLcyFiCTmpfuxFv5)|Sunrise on ISS|
|atlantes_568.jpg|22:08:15|-4.892667|104.524250|Bonglai, Banjit, Way Kanan Regency, Lampung, Indonesia|[Link](https://goo.gl/maps/7FMihUnspsf9Hpgo6)|Crossing Indonesia (at night)|
|atlantes_579.jpg|22:11:00|3.454056|110.452167|South China Sea|[Link](https://goo.gl/maps/39qwLnvBm84TEzLx8)|Sunrise on Earth|
|atlantes_598.jpg|22:15:45|17.696722|121.141833|Katablangan, Conner, Apayao, Philippines|[Link](https://goo.gl/maps/uD1Bp2Wso1iLWcXN9)|Crossing Filipinas|
|atlantes_624.jpg|22:22:15|35.641417|139.452500|Renkōji, Tama, Tokyo, Japan|[Link](https://goo.gl/maps/rq9baVAamvxd9vG28)|Crossing Japan (very cloudy)|
|atlantes_701.jpg|22:41:30|41.288306|-124.100111|Redwoods National Park ASBS State Water Quality Protection Area, California, USA|[Link](https://goo.gl/maps/gwPPCWRfwoWGFyZa7)|Entering North America through California|

We are going to plot again the positions of the ISS on the earth plane but this time indicating with different colors the level of insolation at the time of taking the photo. We will draw the points corresponding to the daytime phase in orange, the sunsets and sunrises in red, and the night phase in blue.

In [ ]:
import os
import pandas
import matplotlib.pyplot as plt
import geopandas

file = os.path.join(path_atlantes, RESULTS_FILE2)     # CSV input file complete path

dataD1 = pandas.read_csv(file, nrows=38).tail(38)                  # Day 1
dataA1 = pandas.read_csv(file, nrows=59).tail(21)                  # Sunset 1
dataN1 = pandas.read_csv(file, nrows=186).tail(127)                # Night 1
dataM1 = pandas.read_csv(file, nrows=208).tail(22)                 # Sunrise 1
dataD2 = pandas.read_csv(file, nrows=409).tail(201)                # Day 2
dataA2 = pandas.read_csv(file, nrows=431).tail(22)                 # Sunset 2
dataN2 = pandas.read_csv(file, nrows=558).tail(127)                # Night 2
dataM2 = pandas.read_csv(file, nrows=579).tail(21)                 # Sunrise 2
dataD3 = pandas.read_csv(file, nrows=713).tail(134)                # Day 3

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

with plt.style.context(("seaborn", "ggplot")):
    world.plot(figsize=(18,10), color="white", edgecolor="grey")
    plt.scatter(dataD1.longitude, dataD1.latitude, zorder=1, alpha= 0.2, c='orange', s=30)
    plt.scatter(dataA1.longitude, dataA1.latitude, zorder=1, alpha= 0.2, c='red', s=30)
    plt.scatter(dataN1.longitude, dataN1.latitude, zorder=1, alpha= 0.2, c='blue', s=30)
    plt.scatter(dataM1.longitude, dataM1.latitude, zorder=1, alpha= 0.2, c='red', s=30)
    plt.scatter(dataD2.longitude, dataD2.latitude, zorder=1, alpha= 0.2, c='orange', s=30)
    plt.scatter(dataA2.longitude, dataA2.latitude, zorder=1, alpha= 0.2, c='red', s=30)
    plt.scatter(dataN2.longitude, dataN2.latitude, zorder=1, alpha= 0.2, c='blue', s=30)
    plt.scatter(dataM2.longitude, dataM2.latitude, zorder=1, alpha= 0.2, c='red', s=30)
    plt.scatter(dataD3.longitude, dataD3.latitude, zorder=1, alpha= 0.2, c='orange', s=30)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title("Insolation conditions: Orange=day; Red=dawn/sunset; Blue=night");

With everything seen so far, it is confirmed that the best series of data is the one that we had indicated before and that it corresponds to the overflight over the American continent, that is, between the pictures `atlantes_307.jpg` and `atlantes_419.jpg`.

## Data processing on pictures

In addition to getting data from most of the SenseHAT sensors, our program took a picture at each iteration of the loop (every 15 seconds). These pictures can also be considered the result of the measurement of a sensor (the camera) and we can extract data from them. A very simple data to obtain and understand is the average of all the values that all the pixels of each picture have. As we know, each pixel has an integer value from 0 to 255 for each of the 3 basic color channels (R: Red, G: Green, B: Blue). We can also average these three values for each pixel to obtain a kind of luminosity value (this is what is usually done to convert images to black and white). So, in total we are going to obtain 4 average values for each picture:

* Average brightness of all pixels in each picture.
* Average R value of all pixels in each picture.
* Average G value of all pixels in each picture.
* Average B value of all pixels in each picture.

In order not to have to be calculating these averages every time, since it is a heavy process (each photo contains millions of pixels), we are going to make a program to preprocess this data and record it in a new CSV file, in a similar way to how we did with the nearest weather stations.

The following program reads the CSV file that ESA gave us with the original data recorded by Raspberry and generates a new CSV file with the calculations made on the pictures of the four averages that we have listed before. The original file is `atlantes.csv` and the new one `atlantes_means.csv`.

In [ ]:
import os
import csv
import pandas
from PIL import Image, ImageStat

file_in = os.path.join(path_atlantes, RESULTS_FILE)       # CSV input file complete path
file_out = os.path.join(path_atlantes, RESULTS_FILE3)     # CSV output file complete path

file = open(file_out, 'w', newline='', encoding='utf-8')  # Opening output file in write mode (`w`)

# Reading specific columns from original file
data = pandas.read_csv(file_in).loc[:, ['datetime', 'picture_file']]

# Writing colum headers to output file
header = ("datetime", "picture_file", "mean_global", "mean_r", "mean_g", "mean_b")
csv.writer(file).writerow(header)

# Iterating data in input file
for index, row in data.iterrows():
    # Loading picture in memory
    im = Image.open(os.path.join(path_atlantes, row[1]))
    # Getting picture statistics, including the mean values of R, G and B channels
    stat = ImageStat.Stat(im)
    # Determining global mean
    mean = round(sum(stat.mean)/len(stat.mean), 2)
    # Writing data to output file
    csv.writer(file).writerow((row[0], row[1], mean, round(stat.mean[0], 2), round(stat.mean[1], 2), round(stat.mean[2], 2)))

# Closing output file
file.close()

# Logging end of process
print("Fichero '%s' generado." % (file_out))

## Chart generation

The CSV file generated in phase 3 by our program executed on board the ISS contains the following columns (and units):

* datetime (AAAA-MM-DD HH:MM:SS.mmm)
* picture_file (atlantes_NNN.jpg)
* latitude (º)
* longitude (º)
* temp_cpu (ºC)
* temp_h (ºC)
* temp_p (ºC)
* humidity (%)
* pressure (mbar)
* pitch (rad)
* roll (rad)
* yaw (rad)
* mag_x (μT)
* mag_y (μT)
* mag_z (μT)
* accel_x (g)
* accel_y (g)
* accel_z (g)
* gyro_x (rad/s)
* gyro_y (rad/s)
* gyro_z (rad/s)

To these columns we can add the means of the pixels of the photos that we have calculated before, that is:

* Global mean (0-255)
* R mean (0-255)
* G mean (0-255)
* B mean (0-255)

With all these data we are going to draw graphs to visually decide which of them may be more suitable to be sonified. During comparisons, two situations can occur:

1. Compare data with the same units, such as `temp_cpu`, `temp_h` and `temp_p`.
2. Compare data with different units, such as `temp_cpu` and `mag_x`.

In the first case we will draw the classic charts with two coordinate axes, X and Y. In this case we can represent as many columns of data as we want.

In the second case we will make charts with two vertical axes, one on each side of the chart. In this case we can only represent two columns of data. We are going to see how to make these two types of charts.

### Charts type 1: Data with same units

For example, we are going to draw the four averages calculated on the pictures at the beginning of this notebook on the same chart. The data will therefore be taken from the CSV file `atlantes_means.csv`. The `datetime` column must be converted to Python datetime objects for the charts library to correctly interpret the dates. This is done by specifying the position of the column in the `parse_dates` parameter of the `read_csv()` function.

In [ ]:
import os
import pandas

file = os.path.join(path_atlantes, RESULTS_FILE3)    # CSV input file complete path

# Reading whole file
data = pandas.read_csv(file, parse_dates=[0])

# Columns in file: "datetime", "picture_file", "mean_global", "mean_r", "mean_g", "mean_b"
data_x = data.datetime                        # Data in 'datetime' column
data_y1 = data.mean_global                    # Data in 'mean_global' column
data_y2 = data.mean_r                         # Data in 'mean_r' column
data_y3 = data.mean_g                         # Data in 'mean_g' column
data_y4 = data.mean_b                         # Data in 'mean_b' column

fig, ax = plt.subplots()                                        # Seting objects to draw with matplotlib
ax.plot(data_x, data_y1, label="Mean global", color="yellow")   # Plotting 'mean_global' against 'datetime'
ax.plot(data_x, data_y2, label="Mean R", color="red")           # Plotting 'mean_r' against 'datetime'
ax.plot(data_x, data_y3, label="Mean G", color="green")         # Plotting 'mean_g' against 'datetime'
ax.plot(data_x, data_y4, label="Mean B", color="blue")          # Plotting 'mean_b' against 'datetime'

plt.legend()                                  # Activating chart legend
plt.grid()                                    # Activating chart grid
plt.show()                                    # Showing the chart

In view of this graph we already see that there is not much difference between the different color channels of the photos. Despite this, we will try to "play" each channel with a different instrument, since it will surely result in a harmonic sound and with those small dissonances or imperfections that occur in an interpretation made by humans.

### Charts type 2: Data with different units

To illustrate this case we are going to represent, for example, the data columns `temp_p` (left axis) and `pressure` (right axis) versus time (thanks to [this article](https://cmdlinetips.com/2019/10/how-to-make-a-plot-with-two-different-y-axis-in-python-with-matplotlib/) for the idea). In this case we will use the CSV file generated by Raspberry directly, that is, the `atlantes.csv`. In the code we indicate all the column headers that we could use.

In [ ]:
import os
import pandas

file = os.path.join(path_atlantes, RESULTS_FILE)    # CSV input file complete path

# Reading whole file
data = pandas.read_csv(file, parse_dates=[0])

# Columns in file:
#     "datetime", "picture_file", "latitude", "longitude", "temp_cpu", "temp_h", "temp_p",
#     "humidity", "pressure", "pitch", "roll", "yaw", "mag_x", "mag_y", "mag_z",
#     "accel_x", "accel_y", "accel_z", "gyro_x", "gyro_y", "gyro_z"

data_x = data.datetime                        # Data in 'datetime' column
data_y1 = data.temp_p                         # Data in 'temp_p' column
data_y2 = data.pressure                       # Data in 'pressure' column

fig, ax = plt.subplots()                      # Seting objects to draw with matplotlib
ax.set_xlabel('Datetime')                     # Setting X axis legend

ax.plot(data_x, data_y1, color="red")         # Plotting 'temp_p' against 'datetime' in red
ax.set_ylabel('Temp. P', color="red")         # Setting Y axis legend

ax2 = ax.twinx()                              # Creating new vertical axis
ax2.plot(data_x, data_y2, color="blue")       # Plotting 'pressure' against 'datetime' in blue
ax2.set_ylabel("Pressure", color="blue")      # Setting second Y axis legend

ax.grid(axis='x')                             # Activating chart grid
plt.show()                                    # Showing the chart

Although we are not analyzing the data yet, in view of the previous graph we cannot fail to emphasize that there is an evident correlation between the temperature and pressure inside the station with the level of insolation, since the two minimums of both curves , correspond fairly roughly with the moments when the Earth eclipsed the Sun in the nearly two complete orbits for which we have data.

With these representation techniques we will be able to perform an analysis of the data in order to identify the best data series to proceed with their sonification.

Next we will deal with the sonification techniques themselves.

## Starting to sonify

To sonify data series, the best tool found is the [sonify](https://github.com/erinspace/sonify) library. Although we are not going to use the original version but a [modified one in Niubit](https://github.com/niubit/sonify) that allows more control over data scaling and let us to save the generated MIDI files. We also modified the tempo to use 60 beats per second which makes it easier to control beats.

A demonstration of the library in action is the [Graphs_and_Sounds](https://github.com/erinspace/sonify/blob/master/examples/Graphs_and_Sounds.ipynb) notebook found inside the `examples` directory of both the [original repository](https://github.com/erinspace/sonify) and [our fork](https://github.com/niubit/sonify). We are going to do the same as seen in that notebook but using our data as a source.

The `sonify` library is not found in [Pypi](https://pypi.org/) so it cannot be installed with the `pip` command. You have to put its source code next to the Jupyter notebook or Python script that will use it. If the first code cell of this notebook has been executed, we will already have the `sonify` library installed.

The sonification will be done with the `create_midi_from_data()` function to which a list of tuples (x, y) must be passed as arguments where `x` is the time in seconds and `y` is the MIDI code of the note to be produced. In the following table we can see the correspondence between the MIDI codes and the real notes:

![MIDI table](https://niubit.net/media/uploads/images/atlantes/midinote2.jpeg)

As we can see, the notes can have an integer value from 0 to 127. We must therefore make sure that the data series that we pass to the `create_midi_from_data()` function contains numbers between 0 and 127 in the second position of each tuple, that is, what we have called `y` before. To facilitate this task there is in the `sonify` library the `function scale_list_to_range(list_to_scale, new_min, new_max)` to which we pass the list of data and the minimum and maximum extremes where we want our list to be "rescaled". That is, the minimum of the `list_to_scale` list will be assigned to `new_min` and the maximum to `new_max`; the rest of the intermediate values in the list will be distributed proportionally between the two extremes. Let's see it with an example:

In [ ]:
import sonify

my_list = [4, 6, 7, 8]

rescaled_list_14_18 = sonify.scale_list_to_range(my_list, 14, 18)
rescaled_list_0_100 = sonify.scale_list_to_range(my_list, 0, 100)

print(rescaled_list_14_18)
print(rescaled_list_0_100)

There is another similar function in our fork of the `sonify` library that allows us more control over the scaling of the data. This is the function `scale_list_to_range2(list_to_scale, new_min, new_max, old_min, old_max)`. It works the same as the one mentioned before, but instead of using the extreme values of the list as the basis for scaling, it allows us to indicate them manually. We see it with an example:

In [ ]:
import sonify

my_list = [4, 6, 7, 8]

rescaled_list_14_18 = sonify.scale_list_to_range2(my_list, 14, 18, 0, 10)
rescaled_list_0_100 = sonify.scale_list_to_range2(my_list, 0, 100, 0, 10)

print(rescaled_list_14_18)
print(rescaled_list_0_100)

### Sonifying one dimension list

Before sonifying the real data, we are going to do a simple test to know the `sonify` library by sonifying the previous list. The `x` value of the tuples that as we know indicates the time in seconds, we will increase it by half a second in each tuple.

In [ ]:
import sonify
import IPython

my_list = [4, 6, 7, 8]
snd_f = "my_list"

# Normalizing the data between 20 and 100 
normalized_list = sonify.scale_list_to_range2(my_list, new_min=20, new_max=100, old_min=0, old_max=10)

final_data = []
x = 0
for i in range(0, len(normalized_list)):
    final_data.append((x, normalized_list[i]))
    x += 0.5

sonify.create_midi_from_data(final_data, file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

### Sonifying multi dimension list

With the `sonify` library we can create polyphonic sound, that is, with several instruments playing simultaneously. To achieve this we only have to pass to the `create_midi_from_data()` function of the `sonify` library a list of tuple lists instead of a simple list of tuples. The first element of each sublist of tuples will be the instrument, which we can choose from the `INSTRUMENTS` dictionary that is inside the `constants.py` file in the `sonify` library (inside the directory that contains its code).

In the previous example, the list of tuples that we passed to `sonify` looked like this:

In [ ]:
my_list = [4, 6, 7, 8]

# Normalizing the data between 20 and 100
normalized_list = sonify.scale_list_to_range2(my_list, new_min=20, new_max=100, old_min=0, old_max=10)

final_data = []
x = 0
for i in range(0, len(normalized_list)):
    final_data.append((x, normalized_list[i]))
    x += 0.5

print(final_data)

Now we need a list of lists (trick: Python's [`zip()`](https://docs.python.org/es/3.8/library/functions.html#zip) function converts two lists to a list of tuples by shuffling the elements of both lists; it actually produces what is called an iterator that we can convert to a list using the [`list()`](https://docs.python.org/es/3.8/library/functions.html#func-list) function). At the beginning of each list of tuples we will add an individual element (not a tuple) with the name of the instrument that will interpret that list (trick: we can add an element to a list with the operator `+`, but both elements to "add" must be lists; to convert an isolated element into a list we will write it in brackets; we can see the trick in action in the last lines of the following code):

In [ ]:
list_y1 = [4, 6, 7, 8]
list_y2 = [1, 3, 1, 3]
list_y3 = [8, 7, 6, 4]

# Normalizing the data between 20 and 100
normalized_y_list1 = sonify.scale_list_to_range2(list_y1, new_min=20, new_max=100, old_min=0, old_max=10)
normalized_y_list2 = sonify.scale_list_to_range2(list_y2, new_min=20, new_max=100, old_min=0, old_max=10)
normalized_y_list3 = sonify.scale_list_to_range2(list_y3, new_min=20, new_max=100, old_min=0, old_max=10)

data_x = []
x = 0
for i in range(0, len(normalized_y_list1)):
    data_x.append(x)
    x += 0.5

list_of_lists = []
list_of_lists.append(['rock organ'] + list(zip(data_x, normalized_y_list1)))
list_of_lists.append(['pizzicato strings'] + list(zip(data_x, normalized_y_list2)))
list_of_lists.append(['oboe'] + list(zip(data_x, normalized_y_list3)))


print(list_of_lists)

The sonification of the list of lists above is as follows:

In [ ]:
snd_f = "list_of_lists"
sonify.create_midi_from_data(list_of_lists, track_type='multiple', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

### Sonifying picture means

We are going to do the next attempt using our data. We are going to sonify the complete series of data in the `mean_global` column that we generate in the section **Data processing on pictures**. The time list will be a sequence in which each item will be incremented by 0.1 so that 10 notes are played per second. We have increased the pace because the data list is much larger than the previous examples, because we don't want the resulting sound to be very long:

In [ ]:
import os
import pandas
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE3)    # CSV file complete path
snd_f = "pictures_mean_global"

# Reading whole file
data = pandas.read_csv(file)

# Columns in file: "datetime", "picture_file", "mean_global", "mean_r", "mean_g", "mean_b"

data_y1 = data.mean_global                    # Data in 'mean_global' column

# Normalizing the data between 20 and 100
data_y1_normalized = sonify.scale_list_to_range2(data_y1, new_min=20, new_max=100, old_min=0, old_max=255)

final_data = []
x = 0
for i in range(0, len(data_y1_normalized)):
    final_data.append((x, data_y1_normalized[i]))
    x += 0.1

sonify.create_midi_from_data(final_data, file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

This is already starting to get rythm.

The following will be similar but with polyphonic sound, using the technique that we have learned before. We are going to use the four columns of means of the pictures that we will associate with four different instruments as follows:

* `mean_global` => steel drums
* `mean_r` => rock organ
* `mean_g` => pizzicato strings
* `mean_b` => oboe

In [ ]:
import os
import pandas
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE3)    # CSV file complete path
snd_f = "pictures_means"

# Reading whole file
data = pandas.read_csv(file)

# Columns in file: "datetime", "picture_file", "mean_global", "mean_r", "mean_g", "mean_b"

data_y1 = data.mean_global                    # Data in 'mean_global' column
data_y1_normalized = sonify.scale_list_to_range2(data_y1, new_min=20, new_max=100, old_min=0, old_max=255)
data_y2 = data.mean_r                         # Data in 'mean_r' column
data_y2_normalized = sonify.scale_list_to_range2(data_y2, new_min=20, new_max=100, old_min=0, old_max=255)
data_y3 = data.mean_g                         # Data in 'mean_g' column
data_y3_normalized = sonify.scale_list_to_range2(data_y3, new_min=20, new_max=100, old_min=0, old_max=255)
data_y4 = data.mean_b                         # Data in 'mean_b' column
data_y4_normalized = sonify.scale_list_to_range2(data_y4, new_min=20, new_max=100, old_min=0, old_max=255)

data_x = []
x = 0
for i in range(0, len(data_y1_normalized)):
    data_x.append(x)
    x += 0.1

multitrack_data = []
multitrack_data.append(list(zip(data_x, data_y1_normalized)))
multitrack_data.append(list(zip(data_x, data_y2_normalized)))
multitrack_data.append(list(zip(data_x, data_y3_normalized)))
multitrack_data.append(list(zip(data_x, data_y4_normalized)))

# Let's add some instruments to each track
instruments_to_add = ['steel drums', 'rock organ', 'pizzicato strings', 'oboe']

multitrack_data_with_instruments = []
for index, track in enumerate(multitrack_data):
    multitrack_data_with_instruments.append([instruments_to_add[index]] + track)

sonify.create_midi_from_data(multitrack_data_with_instruments, track_type='multiple', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

The next thing will be to finally attack the idea of sonifying the weather conditions on the ground at the points where the station flew over, as was our initial objective.

## Data selection

As we saw in the **Weather stations location** section, only when the ISS flies over highly populated continents can we count on the monitoring of weather stations on the ground to be detailed enough to produce a series of data with good resolution so that our project has possibilities.

For this reason, at the end of this section we come to the conclusion that the data series that we are going to use is the one that covers the first overflight over the American continent:

![Fly over america](https://niubit.net/media/uploads/images/atlantes/fly_over_america.png)

Visualizing and analyzing the content of the file `atlantes_stations.csv` we find that those two points correspond to the rows of the following pictures (we are going to use the pictures as an index within the CSV files):

* Begin: `atlantes_307.jpg`
* End: `atlantes_419.jpg`

We are therefore going to interrogate the `meteostat` library so that it offers us the data of some wheater parameters in those locations in the instants of time in which the ISS flew over them. Let's start with the temperature value. As other times, the first thing we will do is preprocess the information, since as we saw in the first tests with the `meteostat` library, the queries are slow, so it is convenient for us to record the information we obtain from it so as not to have to repeat the requests. We will expand the `atlantes_stations.csv` file with a series of new columns where we will store all the data that `meteostat` offers us. Specifically, it allows us to obtain the following weather parameters:

* `temp`: Air temperature in °C
* `dwpt`: Dew point in °C
* `rhum`: Relative humidity in %
* `prcp`: Precipitations in mm/hour
* `snow`: Snow precipitations in mm
* `wdir`: Average wind direction in degrees (º)
* `wspd`: Average wind speed in km/h
* `wpgt`: Maximum speed gusts of wind in km/h
* `pres`: Normalized mean pressure at sea level in hPa
* `tsun`: Minutes of insolation of the last hour in minutes
* `coco`: [Weather conditions code](https://dev.meteostat.net/docs/formats.html#weather-condition-codes)

In [ ]:
import os
import csv
import pandas
import datetime
from meteostat import Hourly

file_in = os.path.join(path_atlantes, RESULTS_FILE2)         # CSV input file complete path
file_out = os.path.join(path_atlantes, RESULTS_FILE4)        # CSV output file complete path

data = pandas.read_csv(file_in, parse_dates=[0])             # Reading whole file

file = open(file_out, 'w', newline='', encoding='utf-8')     # Opening output file in write mode (`w`)

# Writing colum headers to output file
header = ("datetime", "picture_file", "latitude", "longitude", "station_id", "station", "country", "st_lat", "st_lon", "temp", "dwpt", "rhum", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun", "coco")
csv.writer(file).writerow(header)

# Iterating data in input file
for index, row in data.iterrows():
    # Quering the weather station data
    start_date = row["datetime"]
    end_date = row["datetime"] + datetime.timedelta(hours=1)
    station_data = Hourly(row["station_id"], start_date, end_date).fetch()
    
    temp = station_data.temp[0] if len(station_data.temp) > 0 and not math.isnan(station_data.temp[0]) else ""
    dwpt = station_data.dwpt[0] if len(station_data.dwpt) > 0 and not math.isnan(station_data.dwpt[0]) else ""
    rhum = station_data.rhum[0] if len(station_data.rhum) > 0 and not math.isnan(station_data.rhum[0]) else ""
    prcp = station_data.prcp[0] if len(station_data.prcp) > 0 and not math.isnan(station_data.prcp[0]) else ""
    snow = station_data.snow[0] if len(station_data.snow) > 0 and not math.isnan(station_data.snow[0]) else ""
    wdir = station_data.wdir[0] if len(station_data.wdir) > 0 and not math.isnan(station_data.wdir[0]) else ""
    wspd = station_data.wspd[0] if len(station_data.wspd) > 0 and not math.isnan(station_data.wspd[0]) else ""
    wpgt = station_data.wpgt[0] if len(station_data.wpgt) > 0 and not math.isnan(station_data.wpgt[0]) else ""
    pres = station_data.pres[0] if len(station_data.pres) > 0 and not math.isnan(station_data.pres[0]) else ""
    tsun = station_data.tsun[0] if len(station_data.tsun) > 0 and not math.isnan(station_data.tsun[0]) else ""
    coco = station_data.coco[0] if len(station_data.coco) > 0 and not math.isnan(station_data.coco[0]) else ""
    
    # Writing data to output file
    csv.writer(file).writerow((row["datetime"], row["picture_file"], row["latitude"], row["longitude"], row["station_id"], row["station"], row["country"], row["st_lat"], row["st_lon"], temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco))

# Closing output file
file.close()

# Logging end of process
print("Fichero '%s' generado." % (file_out))

Looking at the resulting CSV file we see that `meteostat` provides us with information for most of the locations we have selected, which is a good start.

| datetime                   | picture_file     | temp | dwpt  | rhum  | prcp | snow | wdir  | wspd | wpgt | pres   | tsun | coco |
|----------------------------|------------------|------|-------|-------|------|------|-------|------|------|--------|------|------|
| 2021-04-16 21:03:00.005538 | atlantes_307.jpg | 10.2 | 7.4   | 83.0  | 0.0  |      | 270.0 | 5.4  |      | 1024.9 |      |      |
| 2021-04-16 21:03:15.005907 | atlantes_308.jpg | 25.1 | 4.8   | 27.0  | 0.0  |      | 25.0  | 1.8  |      | 1023.1 |      |      |
| 2021-04-16 21:03:30.006037 | atlantes_309.jpg | 18.0 | 4.2   | 40.0  | 0.0  |      | 60.0  | 7.6  |      | 1023.8 |      | 1.0  |
| 2021-04-16 21:03:45.006218 | atlantes_310.jpg | 13.8 | 8.2   | 69.0  |      |      | 110.0 | 13.0 |      | 1023.5 |      |      |
| 2021-04-16 21:04:00.006389 | atlantes_311.jpg | 23.0 | 2.5   | 26.0  | 0.0  |      | 170.0 | 7.6  |      | 1022.2 |      |      |
| 2021-04-16 21:04:15.006411 | atlantes_312.jpg | 23.0 | -10.3 | 10.0  | 0.0  |      | 220.0 | 5.4  |      | 1023.9 |      |      |
| 2021-04-16 21:04:30.006446 | atlantes_313.jpg | 23.0 | -3.4  | 17.0  | 0.0  |      | 230.0 | 11.0 |      | 1023.0 |      |      |
| 2021-04-16 21:04:45.006630 | atlantes_314.jpg | 21.0 | -5.0  | 17.0  | 0.0  |      | 50.0  | 19.0 |      | 1024.0 |      |      |
| 2021-04-16 21:05:00.006805 | atlantes_315.jpg | 19.0 | -5.9  | 18.0  | 0.0  |      | 360.0 | 19.0 |      | 1024.0 |      |      |
| 2021-04-16 21:05:15.006873 | atlantes_316.jpg | 10.0 | -10.2 | 23.0  | 0.0  |      | 64.0  | 7.9  |      | 1025.0 |      |      |
| 2021-04-16 21:05:30.007029 | atlantes_317.jpg | 15.6 | -5.5  | 23.0  | 0.0  |      | 130.0 | 16.6 |      | 1025.3 |      |      |
| 2021-04-16 21:05:45.007137 | atlantes_318.jpg | 15.6 | -5.5  | 23.0  | 0.0  |      | 130.0 | 16.6 |      | 1025.3 |      |      |
| 2021-04-16 21:06:00.007293 | atlantes_319.jpg | 13.3 | -8.0  | 22.0  | 0.0  |      | 350.0 | 27.7 |      | 1027.6 |      |      |
| 2021-04-16 21:06:15.007372 | atlantes_320.jpg | 9.0  | -8.1  | 29.0  | 0.0  |      | 90.0  | 15.0 |      | 1025.0 |      |      |
| 2021-04-16 21:06:30.007390 | atlantes_321.jpg | 9.0  | -8.1  | 29.0  | 0.0  |      | 90.0  | 15.0 |      | 1025.0 |      |      |
| 2021-04-16 21:06:45.007528 | atlantes_322.jpg | 7.0  | -6.0  | 39.0  | 0.0  |      | 30.0  | 37.0 |      | 1025.0 |      |      |
| 2021-04-16 21:07:00.007552 | atlantes_323.jpg | 2.8  | 0.0   | 82.0  | 0.3  |      | 320.0 | 29.5 |      | 1028.4 |      |      |
| 2021-04-16 21:07:15.007685 | atlantes_324.jpg | 3.3  | -1.1  | 73.0  | 0.3  |      | 350.0 | 16.6 |      | 1025.7 |      |      |
| 2021-04-16 21:07:30.007788 | atlantes_325.jpg | 4.0  | -1.0  | 70.0  | 0.0  |      | 330.0 | 26.0 |      | 1023.0 |      |      |
| 2021-04-16 21:07:45.007964 | atlantes_326.jpg | 2.0  | -0.8  | 82.0  | 0.0  |      | 300.0 | 13.0 |      | 1023.0 |      |      |
| 2021-04-16 21:08:00.008043 | atlantes_327.jpg | 2.0  | -0.8  | 82.0  | 0.0  |      | 300.0 | 13.0 |      | 1023.0 |      |      |
| 2021-04-16 21:08:15.008081 | atlantes_328.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:08:30.008379 | atlantes_329.jpg | 5.0  | 1.7   | 79.0  | 0.5  |      | 330.0 | 29.5 |      | 1021.2 |      |      |
| 2021-04-16 21:08:45.008490 | atlantes_330.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:09:00.008632 | atlantes_331.jpg | 4.0  | 2.7   | 91.0  | 0.5  |      | 340.0 | 20.0 |      | 1019.0 |      |      |
| 2021-04-16 21:09:15.008670 | atlantes_332.jpg | 5.0  | 2.9   | 86.0  | 0.0  |      | 340.0 | 16.6 |      | 1018.2 |      |      |
| 2021-04-16 21:09:30.008734 | atlantes_333.jpg | 7.0  | 4.8   | 86.0  | 0.3  |      | 20.0  | 7.0  |      | 1017.0 |      |      |
| 2021-04-16 21:09:45.008740 | atlantes_334.jpg | 7.0  | 5.5   | 90.0  | 0.6  |      | 10.0  | 13.0 |      | 1016.0 |      |      |
| 2021-04-16 21:10:00.008916 | atlantes_335.jpg | 7.0  | 6.3   | 95.0  | 0.4  |      | 10.0  | 19.0 |      | 1016.0 |      |      |
| 2021-04-16 21:10:15.008948 | atlantes_336.jpg | 10.0 | 8.9   | 93.0  | 0.8  |      | 50.0  | 6.0  |      | 1014.0 |      |      |
| 2021-04-16 21:10:30.009012 | atlantes_337.jpg | 10.0 | 10.0  | 100.0 | 0.5  |      | 60.0  | 6.0  |      | 1014.0 |      | 5.0  |
| 2021-04-16 21:10:45.009113 | atlantes_338.jpg | 11.0 | 9.3   | 89.0  | 0.1  |      | 340.0 | 6.0  |      | 1014.0 |      |      |
| 2021-04-16 21:11:00.009115 | atlantes_339.jpg | 13.0 | 9.1   | 77.0  | 0.0  |      | 95.0  | 8.3  |      | 1013.0 |      |      |
| 2021-04-16 21:11:15.009173 | atlantes_340.jpg | 15.0 | 4.9   | 51.0  | 0.0  |      | 60.0  | 6.0  |      | 1012.0 |      |      |
| 2021-04-16 21:11:30.009298 | atlantes_341.jpg | 15.0 | 7.1   | 59.0  | 0.0  |      | 50.0  | 6.0  |      | 1012.0 |      |      |
| 2021-04-16 21:11:45.009406 | atlantes_342.jpg | 16.0 | 7.0   | 55.0  | 0.0  |      | 100.0 | 6.0  |      | 1012.0 |      |      |
| 2021-04-16 21:12:00.009458 | atlantes_343.jpg | 17.0 | 8.2   | 56.0  | 0.0  |      | 346.0 | 3.2  |      | 1012.0 |      |      |
| 2021-04-16 21:12:15.010704 | atlantes_344.jpg | 19.4 | 7.8   | 47.0  | 0.0  |      | 338.0 | 2.2  |      | 1010.5 |      |      |
| 2021-04-16 21:12:30.010722 | atlantes_345.jpg | 19.0 | 8.3   | 50.0  | 0.0  |      | 350.0 | 7.0  |      | 1011.0 |      |      |
| 2021-04-16 21:12:45.010722 | atlantes_346.jpg | 19.0 | 12.3  | 65.0  | 0.0  |      | 61.0  | 6.5  |      | 1011.0 |      |      |
| 2021-04-16 21:13:00.010827 | atlantes_347.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:13:15.011000 | atlantes_348.jpg | 18.3 | 18.3  | 100.0 | 1.3  |      | 80.0  | 13.0 |      | 1009.3 |      |      |
| 2021-04-16 21:13:30.011110 | atlantes_349.jpg | 18.0 | 17.7  | 98.0  | 3.0  |      | 110.0 | 5.4  |      | 1011.0 |      |      |
| 2021-04-16 21:13:45.011155 | atlantes_350.jpg | 24.0 | 20.9  | 83.0  | 0.0  |      | 240.0 | 13.0 |      | 1011.0 |      |      |
| 2021-04-16 21:14:00.011237 | atlantes_351.jpg | 25.8 | 19.4  | 68.0  | 0.0  |      | 244.0 | 17.3 |      | 1010.5 |      |      |
| 2021-04-16 21:14:15.012541 | atlantes_352.jpg | 26.0 | 18.9  | 65.0  | 0.0  |      | 240.0 | 17.0 |      | 1012.0 |      |      |
| 2021-04-16 21:14:30.012680 | atlantes_353.jpg | 27.0 | 18.0  | 58.0  | 0.0  |      | 260.0 | 9.0  |      | 1011.0 |      |      |
| 2021-04-16 21:14:45.012855 | atlantes_354.jpg | 27.0 | 17.8  | 57.0  | 0.0  |      | 180.0 | 13.0 |      | 1011.0 |      |      |
| 2021-04-16 21:15:00.012925 | atlantes_355.jpg | 25.4 | 20.9  | 76.0  | 0.0  |      | 174.0 | 25.6 |      | 1010.6 |      |      |
| 2021-04-16 21:15:15.013805 | atlantes_356.jpg | 25.4 | 20.9  | 76.0  | 0.0  |      | 174.0 | 25.6 |      | 1010.6 |      |      |
| 2021-04-16 21:15:30.013831 | atlantes_357.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:15:45.013968 | atlantes_358.jpg | 33.0 | 15.9  | 36.0  | 0.0  |      | 180.0 | 9.4  |      | 1012.0 |      |      |
| 2021-04-16 21:16:00.013978 | atlantes_359.jpg | 33.0 | 15.9  | 36.0  | 0.0  |      | 180.0 | 9.4  |      | 1012.0 |      |      |
| 2021-04-16 21:16:15.014030 | atlantes_360.jpg | 31.0 | 11.9  | 31.0  | 0.0  |      | 30.0  | 25.9 |      | 1012.0 |      |      |
| 2021-04-16 21:16:30.014085 | atlantes_361.jpg | 31.0 | 11.9  | 31.0  | 0.0  |      | 30.0  | 25.9 |      | 1012.0 |      |      |
| 2021-04-16 21:16:45.014247 | atlantes_362.jpg | 28.9 | 22.2  | 67.0  | 0.0  |      | 230.0 | 25.9 |      | 1012.3 |      |      |
| 2021-04-16 21:17:00.014254 | atlantes_363.jpg | 28.9 | 22.2  | 67.0  | 0.0  |      | 230.0 | 25.9 |      | 1012.3 |      |      |
| 2021-04-16 21:17:15.014262 | atlantes_364.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:17:30.014365 | atlantes_365.jpg | 28.0 | 24.0  | 79.0  | 0.0  |      | 130.0 | 25.9 |      | 1012.0 |      |      |
| 2021-04-16 21:17:45.014532 | atlantes_366.jpg | 28.0 | 24.0  | 79.0  | 0.0  |      | 130.0 | 25.9 |      | 1012.0 |      |      |
| 2021-04-16 21:18:00.014589 | atlantes_367.jpg | 28.0 | 24.0  | 79.0  | 0.0  |      | 130.0 | 25.9 |      | 1012.0 |      |      |
| 2021-04-16 21:18:15.014705 | atlantes_368.jpg | 28.0 | 24.0  | 79.0  | 0.0  |      | 130.0 | 25.9 |      | 1012.0 |      |      |
| 2021-04-16 21:18:30.014789 | atlantes_369.jpg | 29.0 | 24.1  | 75.0  | 0.0  |      | 80.0  | 40.7 |      | 1009.0 |      | 1.0  |
| 2021-04-16 21:18:45.014814 | atlantes_370.jpg | 29.0 | 24.1  | 75.0  | 0.0  |      | 80.0  | 40.7 |      | 1009.0 |      | 1.0  |
| 2021-04-16 21:19:00.014908 | atlantes_371.jpg | 29.0 | 24.1  | 75.0  | 0.0  |      | 80.0  | 40.7 |      | 1009.0 |      | 1.0  |
| 2021-04-16 21:19:15.015082 | atlantes_372.jpg | 28.0 | 22.9  | 74.0  | 0.0  |      | 80.0  | 24.1 |      | 1009.0 |      | 2.0  |
| 2021-04-16 21:19:30.015442 | atlantes_373.jpg | 28.5 | 23.9  | 76.0  | 0.0  |      | 77.0  | 32.4 |      | 1009.5 |      | 3.0  |
| 2021-04-16 21:19:45.015607 | atlantes_374.jpg | 26.0 | 22.1  | 79.0  | 0.0  |      | 95.0  | 4.0  |      | 1009.0 |      |      |
| 2021-04-16 21:20:00.015665 | atlantes_375.jpg | 28.5 | 20.8  | 63.0  | 0.3  |      | 150.0 | 4.7  |      | 1009.8 |      |      |
| 2021-04-16 21:20:15.015812 | atlantes_376.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:20:30.015970 | atlantes_377.jpg | 30.2 | 22.1  | 62.0  | 0.0  |      | 121.0 | 4.7  |      | 1007.9 |      |      |
| 2021-04-16 21:20:45.016121 | atlantes_378.jpg | 30.2 | 22.1  | 62.0  | 0.0  |      | 121.0 | 4.7  |      | 1007.9 |      |      |
| 2021-04-16 21:21:00.016172 | atlantes_379.jpg | 30.6 | 21.1  | 57.0  | 0.0  |      | 87.0  | 12.2 |      | 1007.8 |      |      |
| 2021-04-16 21:21:15.016223 | atlantes_380.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:21:30.016978 | atlantes_381.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:21:45.017138 | atlantes_382.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:22:00.017232 | atlantes_383.jpg | 22.2 | 19.0  | 82.0  | 0.0  |      | 96.0  | 7.9  |      | 1011.0 |      |      |
| 2021-04-16 21:22:15.017427 | atlantes_384.jpg | 22.2 | 19.0  | 82.0  | 0.0  |      | 96.0  | 7.9  |      | 1011.0 |      |      |
| 2021-04-16 21:22:30.017556 | atlantes_385.jpg | 24.4 | 24.2  | 99.0  | 0.0  |      | 334.0 | 2.5  |      | 1008.3 |      |      |
| 2021-04-16 21:22:45.017673 | atlantes_386.jpg | 24.4 | 24.2  | 99.0  | 0.0  |      | 334.0 | 2.5  |      | 1008.3 |      |      |
| 2021-04-16 21:23:00.017681 | atlantes_387.jpg | 24.4 | 24.2  | 99.0  | 0.0  |      | 334.0 | 2.5  |      | 1008.3 |      |      |
| 2021-04-16 21:23:15.017756 | atlantes_388.jpg | 24.4 | 24.2  | 99.0  | 0.0  |      | 334.0 | 2.5  |      | 1008.3 |      |      |
| 2021-04-16 21:23:30.017828 | atlantes_389.jpg | 25.8 | 23.5  | 87.0  | 0.0  |      | 30.0  | 7.6  |      | 1007.5 |      |      |
| 2021-04-16 21:23:45.017968 | atlantes_390.jpg | 25.8 | 23.5  | 87.0  | 0.0  |      | 30.0  | 7.6  |      | 1007.5 |      |      |
| 2021-04-16 21:24:00.018016 | atlantes_391.jpg | 26.1 | 23.8  | 87.0  | 0.0  |      | 26.0  | 7.9  |      | 1007.4 |      |      |
| 2021-04-16 21:24:15.018450 | atlantes_392.jpg | 27.6 | 24.2  | 82.0  | 0.0  |      | 80.0  | 5.8  |      | 1007.1 |      |      |
| 2021-04-16 21:24:30.018555 | atlantes_393.jpg | 26.8 | 24.1  | 85.0  | 0.1  |      | 78.0  | 7.2  |      | 1007.5 |      |      |
| 2021-04-16 21:24:45.018556 | atlantes_394.jpg | 26.8 | 24.1  | 85.0  | 0.1  |      | 78.0  | 7.2  |      | 1007.5 |      |      |
| 2021-04-16 21:25:00.018616 | atlantes_395.jpg | 26.4 | 24.4  | 89.0  | 0.1  |      | 66.0  | 9.4  |      | 1008.0 |      |      |
| 2021-04-16 21:25:15.018671 | atlantes_396.jpg | 26.4 | 24.4  | 89.0  | 0.1  |      | 66.0  | 9.4  |      | 1008.0 |      |      |
| 2021-04-16 21:25:30.018742 | atlantes_397.jpg | 27.5 | 23.5  | 79.0  |      |      | 62.0  | 5.5  |      | 1009.0 |      | 3.0  |
| 2021-04-16 21:25:45.018763 | atlantes_398.jpg | 23.3 | 22.3  | 94.0  | 0.5  |      | 86.0  | 11.2 |      | 1010.0 |      |      |
| 2021-04-16 21:26:00.018784 | atlantes_399.jpg | 23.3 | 22.3  | 94.0  | 0.5  |      | 86.0  | 11.2 |      | 1010.0 |      |      |
| 2021-04-16 21:26:15.018791 | atlantes_400.jpg | 23.3 | 22.3  | 94.0  | 0.5  |      | 86.0  | 11.2 |      | 1010.0 |      |      |
| 2021-04-16 21:26:30.018937 | atlantes_401.jpg | 25.1 | 23.0  | 88.0  | 0.3  |      | 96.0  | 8.6  |      | 1009.5 |      |      |
| 2021-04-16 21:26:45.018989 | atlantes_402.jpg | 25.1 | 23.0  | 88.0  | 0.3  |      | 96.0  | 8.6  |      | 1009.5 |      |      |
| 2021-04-16 21:27:00.019069 | atlantes_403.jpg | 25.1 | 22.4  | 75.0  | 0.0  |      | 356.0 | 3.6  |      | 1009.6 |      |      |
| 2021-04-16 21:27:15.019124 | atlantes_404.jpg | 25.1 | 22.4  | 75.0  | 0.0  |      | 356.0 | 3.6  |      | 1009.6 |      |      |
| 2021-04-16 21:27:30.019154 | atlantes_405.jpg | 25.1 | 22.4  | 75.0  | 0.0  |      | 356.0 | 3.6  |      | 1009.6 |      |      |
| 2021-04-16 21:27:45.019162 | atlantes_406.jpg | 25.1 | 22.4  | 75.0  | 0.0  |      | 356.0 | 3.6  |      | 1009.6 |      |      |
| 2021-04-16 21:28:00.019205 | atlantes_407.jpg | 25.0 | 19.4  | 71.0  | 0.0  |      | 272.0 | 0.0  |      | 1009.7 |      |      |
| 2021-04-16 21:28:15.020610 | atlantes_408.jpg | 25.0 | 19.4  | 71.0  | 0.0  |      | 272.0 | 0.0  |      | 1009.7 |      |      |
| 2021-04-16 21:28:30.020769 | atlantes_409.jpg | 24.5 | 16.2  | 60.0  | 0.0  |      | 82.0  | 5.8  |      | 1010.2 |      |      |
| 2021-04-16 21:28:45.020903 | atlantes_410.jpg |      |       |       |      |      |       |      |      |        |      |      |
| 2021-04-16 21:29:00.021024 | atlantes_411.jpg | 20.0 | 13.9  | 68.0  | 0.0  |      | 81.0  | 6.5  |      | 1014.3 |      |      |
| 2021-04-16 21:29:15.021039 | atlantes_412.jpg | 21.0 | 14.2  | 65.0  | 0.0  |      | 77.0  | 5.8  |      | 1012.8 |      |      |
| 2021-04-16 21:29:30.021189 | atlantes_413.jpg | 21.1 | 12.8  | 59.0  | 0.0  |      | 95.0  | 7.4  |      | 1011.9 |      | 2.0  |
| 2021-04-16 21:29:45.021263 | atlantes_414.jpg | 21.1 | 12.8  | 59.0  | 0.0  |      | 95.0  | 7.4  |      | 1011.9 |      | 2.0  |
| 2021-04-16 21:30:00.021401 | atlantes_415.jpg | 22.0 | 12.0  | 53.0  | 0.0  |      | 77.0  | 8.3  |      | 1013.6 |      |      |
| 2021-04-16 21:30:15.021410 | atlantes_416.jpg | 18.0 | 9.6   | 58.0  | 0.0  |      | 91.0  | 10.1 |      | 1015.7 |      |      |
| 2021-04-16 21:30:30.021508 | atlantes_417.jpg | 17.0 | 11.1  | 68.0  | 0.0  |      | 70.0  | 7.6  |      | 1021.0 |      |      |
| 2021-04-16 21:30:45.021579 | atlantes_418.jpg | 18.2 | 12.4  | 69.0  | 0.0  |      | 73.0  | 10.1 |      | 1017.3 |      |      |
| 2021-04-16 21:31:00.022159 | atlantes_419.jpg | 14.7 | 11.5  | 81.0  | 0.0  |      | 2.0   | 5.0  |      | 1017.4 |      |      |

## Sonification 1: temperature on ISS vs Earth

The objective of our project was to compare the stability of the environmental conditions on board the station with those very rapidly changing at the same time in the projection on the surface of the Earth. We already have the data at our fingertips. We are going to represent both temperatures in the flyby of America. We will approximate the missing data in the Earth data file by taking the previous value in the table.

In [ ]:
import os
import pandas
import math

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
file_earth = os.path.join(path_atlantes, RESULTS_FILE4)    # CSV Earth file complete path

data_iss = pandas.read_csv(file_iss, nrows=419, parse_dates=[0]).tail(113)        # Reading only rows between 307 and 419
data_earth = pandas.read_csv(file_earth, nrows=419, parse_dates=[0]).tail(113)    # Reading only rows between 307 and 419

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
# Columns in Earth file: datetime, picture_file, temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco
data_x = data_iss.datetime                    # Data in 'datetime' column
data_y1 = data_iss.temp_p                     # Data in 'temp_p' column
data_y2 = data_earth.temp                     # Data in 'temp' column
data_y2 = list(data_y2)

# Filling missing data
last_y2 = data_y2[0]
for i in range(0, len(data_y2)):
    if not math.isnan(data_y2[i]):
        last_y2 = data_y2[i]
    data_y2[i] = last_y2

fig, ax = plt.subplots()                      # Seting objects to draw with matplotlib
ax.plot(data_x, data_y1, label="Temp ISS")    # Plotting 'temp_p' against 'datetime'
ax.plot(data_x, data_y2, label="Temp Earth")  # Plotting 'temp' against 'datetime'

plt.legend()                                  # Activating chart legend
plt.grid()                                    # Activating chart grid
plt.show()                                    # Showing the chart

Unsurprisingly, the graph above shows the station's advantage in flying above the atmosphere, allowing it to avoid rapidly changing conditions at the speed at which it is moving over the earth's surface. Surely the actual temperature values on the ISS are lower than those obtained with the temperature sensor of Raspberry Pi, which will be affected by its own internal heat. Although the variation is credible. In fact, as we saw in the section **Charts type 2: Data with different units**, there seems to be a correlation of the slight variation registered by the sensor with the degree of insolation received by the station.

We are going to sonify the two previous data series.

In [ ]:
import os
import pandas
import sonify
import IPython

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
file_earth = os.path.join(path_atlantes, RESULTS_FILE4)    # CSV Earth file complete path

data_iss = pandas.read_csv(file_iss, nrows=419, parse_dates=[0]).tail(113)        # Reading only rows between 307 and 419
data_earth = pandas.read_csv(file_earth, nrows=419, parse_dates=[0]).tail(113)    # Reading only rows between 307 and 419

snd_f = "temperatures_iss_earth"

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
# Columns in Earth file: datetime, picture_file, temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco
data_x = data_iss.datetime                    # Data in 'datetime' column
data_y1 = data_iss.temp_p                     # Data in 'temp_p' column
data_y1_normalized = sonify.scale_list_to_range2(data_y1, new_min=20, new_max=100, old_min=0, old_max=35)
data_y2 = data_earth.temp                     # Data in 'temp' column
data_y2_normalized = sonify.scale_list_to_range2(data_y2, new_min=20, new_max=100, old_min=0, old_max=35)

data_x = []
x = 0
last_y1 = data_y1_normalized[0]
last_y2 = data_y2_normalized[0]
for i in range(0, len(data_y1_normalized)):
    data_x.append(x)
    if not math.isnan(data_y1_normalized[i]):
        last_y1 = data_y1_normalized[i]
    data_y1_normalized[i] = last_y1
    if not math.isnan(data_y2_normalized[i]):
        last_y2 = data_y2_normalized[i]
    data_y2_normalized[i] = last_y2
    x += 0.2

multitrack_data = []
multitrack_data.append(list(zip(data_x, data_y1_normalized)))
multitrack_data.append(list(zip(data_x, data_y2_normalized)))

# Let's add some instruments to each track
instruments_to_add = ['acoustic grand piano', 'church organ']

multitrack_data_with_instruments = []
for index, track in enumerate(multitrack_data):
    multitrack_data_with_instruments.append([instruments_to_add[index]] + track)

sonify.create_midi_from_data(multitrack_data_with_instruments, track_type='multiple', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f + ".wav")

In the sonification above, the temperature of the ISS is "interpreted" by the hammering piano and that of the ground surface with an organ. It is clearly observed how the note of the piano is practically constant while the organ varies the tone continuously.

This really was our ultimate goal, so we might consider our work finished. Now we would like to compose a video with the photos taken of the Earth's surface and the sound we just created.

Before beginning we have to prepare the photos by copying the ones we have selected (from `atlantes_307.jpg` to `atlantes_419.jpg`) and renaming them with a number from `001` to `113`. We have already made that selection/rename on the `atlantes_fly_over_america` directory. To make the video, it only remains to execute the following cell. In it, a video is first created with the series of photos and then the previous sonification is added as a soundtrack (it costs a bit; the video player in Firefox gives a MIME types error so use Chrome):

In [ ]:
from IPython.display import HTML
from base64 import b64encode

!ffmpeg -y -loglevel quiet -f image2 -r 5 -i atlantes_fly_over_america/%03d.jpg -vf scale=320:240 -r:v 5 -c:v libx264 -qp 0 -preset veryslow -an fly_over_america_video.mp4
!ffmpeg -y -loglevel quiet -i fly_over_america_video.mp4 -i temperatures_iss_earth.wav -c:v copy -c:a aac temperatures_iss_earth.mp4

mp4 = open('temperatures_iss_earth.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The same video with high resolution on YouTube:

In [ ]:
import IPython

IPython.display.YouTubeVideo('Hl1exe3S_cI')

Observing the final result, it seems to be detected how the temperature when flying over the sea is generally higher than on land (it is detected by the highest notes of the organ). It is also detected that in general the temperature in South America is higher than in North America.

At this point we would have already achieved our main objective. From now on, we are going to take advantage of the fact that we have mastered the techniques to sonify data series and to create stopmotion videos with pictures, to try to achieve more effective sonifications.

We are going to try more sonifications with the data we have to see if we are lucky and some of them are moderately harmonic.

The result of sonifying several of the columns of data from the overflight over the American continent (specifically temp, dwpt, rhum, wspd and pres) was too cacophonic as can be seen below:

In [ ]:
import IPython

IPython.display.Audio("temp_dwpt_rhum_wspd_pres.wav")

So we decided to continue with the strategy of sonifying the data separately as we did with temperature.

The only comparable data between ISS and Earth besides the temperature that we have already used, are:

* humidity
* pressure

So we are going to repeat the process done with the temperature but this time with humidity and pressure.

## Sonification 2: relative humidity on ISS vs Earth

We start by making a graphic representation of what we are going to sonify:

In [ ]:
import os
import pandas
import math

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
file_earth = os.path.join(path_atlantes, RESULTS_FILE4)    # CSV Earth file complete path

data_iss = pandas.read_csv(file_iss, nrows=419, parse_dates=[0]).tail(113)        # Reading only rows between 307 and 419
data_earth = pandas.read_csv(file_earth, nrows=419, parse_dates=[0]).tail(113)    # Reading only rows between 307 and 419

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
# Columns in Earth file: datetime, picture_file, temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco
data_x = data_iss.datetime                    # Data in 'datetime' column
data_y1 = data_iss.humidity                   # Data in 'humidity' column
data_y2 = data_earth.rhum                     # Data in 'rhum' column
data_y2 = list(data_y2)

# Filling missing data
last_y2 = data_y2[0]
for i in range(0, len(data_y2)):
    if not math.isnan(data_y2[i]):
        last_y2 = data_y2[i]
    data_y2[i] = last_y2

fig, ax = plt.subplots()                      # Seting objects to draw with matplotlib
ax.plot(data_x, data_y1, label="Humedad relativa ISS")    # Plotting 'humidity' against 'datetime'
ax.plot(data_x, data_y2, label="Humedad relativa Earth")  # Plotting 'rhum' against 'datetime'

plt.legend()                                  # Activating chart legend
plt.grid()                                    # Activating chart grid
plt.show()                                    # Showing the chart

Finally we sonify:

In [ ]:
import os
import pandas
import sonify
import IPython

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
file_earth = os.path.join(path_atlantes, RESULTS_FILE4)    # CSV Earth file complete path

data_iss = pandas.read_csv(file_iss, nrows=419, parse_dates=[0]).tail(113)        # Reading only rows between 307 and 419
data_earth = pandas.read_csv(file_earth, nrows=419, parse_dates=[0]).tail(113)    # Reading only rows between 307 and 419

snd_f = "humidity_iss_earth"

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
# Columns in Earth file: datetime, picture_file, temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco
data_x = data_iss.datetime                                 # Data in 'datetime' column
data_y1 = data_iss.humidity                                # Data in 'humidity' column
data_y1_normalized = sonify.scale_list_to_range2(data_y1, new_min=20, new_max=100, old_min=0, old_max=100)
data_y2 = data_earth.rhum                                  # Data in 'rhum' column
data_y2_normalized = sonify.scale_list_to_range2(data_y2, new_min=20, new_max=100, old_min=0, old_max=100)

data_x = []
x = 0
last_y1 = data_y1_normalized[0]
last_y2 = data_y2_normalized[0]
for i in range(0, len(data_y1_normalized)):
    data_x.append(x)
    if not math.isnan(data_y1_normalized[i]):
        last_y1 = data_y1_normalized[i]
    data_y1_normalized[i] = last_y1
    if not math.isnan(data_y2_normalized[i]):
        last_y2 = data_y2_normalized[i]
    data_y2_normalized[i] = last_y2
    x += 0.2

multitrack_data = []
multitrack_data.append(list(zip(data_x, data_y1_normalized)))
multitrack_data.append(list(zip(data_x, data_y2_normalized)))

# Let's add some instruments to each track
instruments_to_add = ['acoustic grand piano', 'church organ']

multitrack_data_with_instruments = []
for index, track in enumerate(multitrack_data):
    multitrack_data_with_instruments.append([instruments_to_add[index]] + track)

sonify.create_midi_from_data(multitrack_data_with_instruments, track_type='multiple', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f + ".wav")

The result is still not very spectacular and similar to that obtained with temperatures.

## Sonification 3: pressure on ISS vs Tierra

We start by making a graphic representation of what we are going to sonify:

In [ ]:
import os
import pandas
import math

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
file_earth = os.path.join(path_atlantes, RESULTS_FILE4)    # CSV Earth file complete path

data_iss = pandas.read_csv(file_iss, nrows=419, parse_dates=[0]).tail(113)        # Reading only rows between 307 and 419
data_earth = pandas.read_csv(file_earth, nrows=419, parse_dates=[0]).tail(113)    # Reading only rows between 307 and 419

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
# Columns in Earth file: datetime, picture_file, temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco
data_x = data_iss.datetime                                 # Data in 'datetime' column
data_y1 = data_iss.pressure                                # Data in 'pressure' column
data_y2 = data_earth.pres                                  # Data in 'pres' column
data_y2 = list(data_y2)

# Filling missing data
last_y2 = data_y2[0]
for i in range(0, len(data_y2)):
    if not math.isnan(data_y2[i]):
        last_y2 = data_y2[i]
    data_y2[i] = last_y2

fig, ax = plt.subplots()                         # Seting objects to draw with matplotlib
ax.plot(data_x, data_y1, label="Presión ISS")    # Plotting 'pressure' against 'datetime'
ax.plot(data_x, data_y2, label="Presión Earth")  # Plotting 'pres' against 'datetime'

plt.legend()                                  # Activating chart legend
plt.grid()                                    # Activating chart grid
plt.show()                                    # Showing the chart

Sonifying...

In [ ]:
import os
import pandas
import sonify
import IPython

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
file_earth = os.path.join(path_atlantes, RESULTS_FILE4)    # CSV Earth file complete path

data_iss = pandas.read_csv(file_iss, nrows=419, parse_dates=[0]).tail(113)        # Reading only rows between 307 and 419
data_earth = pandas.read_csv(file_earth, nrows=419, parse_dates=[0]).tail(113)    # Reading only rows between 307 and 419

snd_f = "pressure_iss_earth"

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
# Columns in Earth file: datetime, picture_file, temp, dwpt, rhum, prcp, snow, wdir, wspd, wpgt, pres, tsun, coco
data_x = data_iss.datetime                                 # Data in 'datetime' column
data_y1 = data_iss.pressure                                # Data in 'pressure' column
data_y1_normalized = sonify.scale_list_to_range2(data_y1, new_min=20, new_max=100, old_min=980, old_max=1030)
data_y2 = data_earth.pres                                  # Data in 'pres' column
data_y2_normalized = sonify.scale_list_to_range2(data_y2, new_min=20, new_max=100, old_min=980, old_max=1030)

data_x = []
x = 0
last_y1 = data_y1_normalized[0]
last_y2 = data_y2_normalized[0]
for i in range(0, len(data_y1_normalized)):
    data_x.append(x)
    if not math.isnan(data_y1_normalized[i]):
        last_y1 = data_y1_normalized[i]
    data_y1_normalized[i] = last_y1
    if not math.isnan(data_y2_normalized[i]):
        last_y2 = data_y2_normalized[i]
    data_y2_normalized[i] = last_y2
    x += 0.2

multitrack_data = []
multitrack_data.append(list(zip(data_x, data_y1_normalized)))
multitrack_data.append(list(zip(data_x, data_y2_normalized)))

# Let's add some instruments to each track
instruments_to_add = ['acoustic grand piano', 'church organ']

multitrack_data_with_instruments = []
for index, track in enumerate(multitrack_data):
    multitrack_data_with_instruments.append([instruments_to_add[index]] + track)

sonify.create_midi_from_data(multitrack_data_with_instruments, track_type='multiple', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f + ".wav")

Again, nothing groundbreaking.

## Sonification 4: magnetometer components on ISS

We are going to change strategy. Representing graphically all the data captured in the station by the program executed in AstroPi, it is found that the only magnitudes that produce a continuous spectrum of values and whose variation may be of any interest are:

* yaw
* mag_x
* mag_y
* mag_z

Since the only ones that make sense to handle together are the last three, we are going to focus on them. We begin by plotting its variation. This time we are going to use the whole dataset, not just the flyby over America.

In [ ]:
import os
import pandas
import math

file_iss = os.path.join(path_atlantes, RESULTS_FILE)       # CSV ISS file complete path
data_iss = pandas.read_csv(file_iss, parse_dates=[0])      # Reading whole file

# Columns in ISS file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
data_x = data_iss.datetime
data_y1 = data_iss.mag_x
data_y2 = data_iss.mag_y
data_y3 = data_iss.mag_z

fig, ax = plt.subplots()
ax.plot(data_x, data_y1, label="mag_x")
ax.plot(data_x, data_y2, label="mag_y")
ax.plot(data_x, data_y3, label="mag_z")

plt.legend()
plt.grid()
plt.show()

The sonification is as follows. This time we have accelerated the rhythm (one note every tenth of a second) because all the data was used:

In [ ]:
import os
import pandas
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE)    # CSV ISS file complete path
snd_f = "mag_xyz"

data = pandas.read_csv(file, parse_dates=[0])       # Reading whole file

# Columns in file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
data_x = data.datetime                              # Data in 'datetime' column
data_y1 = data.mag_x                                # Data in 'mag_x' column
data_y1_normalized = sonify.scale_list_to_range2(data_y1, new_min=20, new_max=100, old_min=-20, old_max=70)
data_y2 = data.mag_y                                # Data in 'mag_y' column
data_y2_normalized = sonify.scale_list_to_range2(data_y2, new_min=20, new_max=100, old_min=-20, old_max=70)
data_y3 = data.mag_z                                # Data in 'mag_z' column
data_y3_normalized = sonify.scale_list_to_range2(data_y3, new_min=20, new_max=100, old_min=-20, old_max=70)

data_x = []
x = 0
for i in range(0, len(data_y1_normalized)):
    data_x.append(x)
    x += 0.1


multitrack_data = []
multitrack_data.append(list(zip(data_x, data_y1_normalized)))
multitrack_data.append(list(zip(data_x, data_y2_normalized)))
multitrack_data.append(list(zip(data_x, data_y3_normalized)))


# Let's add some instruments to each track
instruments_to_add = ['church organ', 'accordion', 'cello']

multitrack_data_with_instruments = []
for index, track in enumerate(multitrack_data):
    multitrack_data_with_instruments.append([instruments_to_add[index]] + track)


sonify.create_midi_from_data(multitrack_data_with_instruments, track_type='multiple', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

We create the stopmotion video with this sonification (the execution of the following cell takes a while):

In [ ]:
from IPython.display import HTML
from base64 import b64encode

!ffmpeg -y -loglevel quiet -f image2 -r 10 -i atlantes/atlantes_%03d.jpg -vf scale=320:240 -r:v 10 -c:v libx264 -qp 0 -preset veryslow -an complete_fly_video.mp4
!ffmpeg -y -loglevel quiet -i complete_fly_video.mp4 -i mag_xyz.wav -c:v copy -c:a aac mag_xyz.mp4

mp4 = open('mag_xyz.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The same video with high resolution on YouTube:

In [ ]:
import IPython

IPython.display.YouTubeVideo('VSILkRGMKdA')

## Sonification 5: magnetometer modulus on ISS

We are going to calculate the modulus of the vector composed of the magnitudes `mag_x`,` mag_y` and `mag_z`, to see how the intensity of the Earth's magnetic field varies during the orbit of the ISS. The modulus is calculated with the following function:

$Mag = \sqrt{mag_x^2 + mag_y^2 + mag_z^2}$

We start by representing this modulus graphically:

In [ ]:
import os
import pandas
import numpy
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE)    # CSV ISS file complete path
snd_f = "mag_xyz"

data = pandas.read_csv(file, parse_dates=[0])       # Reading whole file

# Columns in file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
data_x = data.datetime                              # Data in 'datetime' column
data_y1 = data.mag_x                                # Data in 'mag_x' column
data_y2 = data.mag_y                                # Data in 'mag_y' column
data_y3 = data.mag_z                                # Data in 'mag_z' column

data_y = numpy.sqrt(data_y1.pow(2) + data_y2.pow(2) + data_y3.pow(2))

fig, ax = plt.subplots()
ax.plot(data_x, data_y, label="Mag")

plt.legend()
plt.grid()
plt.show()

The sonification is as follows.

In [ ]:
import os
import pandas
import numpy
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE)    # CSV ISS file complete path
snd_f = "mag"

data = pandas.read_csv(file, parse_dates=[0])       # Reading whole file

# Columns in file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
data_x = data.datetime                              # Data in 'datetime' column
data_y1 = data.mag_x                                # Data in 'mag_x' column
data_y2 = data.mag_y                                # Data in 'mag_y' column
data_y3 = data.mag_z                                # Data in 'mag_z' column

data_y = numpy.sqrt(data_y1.pow(2) + data_y2.pow(2) + data_y3.pow(2))

normalized_list = sonify.scale_list_to_range(data_y, new_min=20, new_max=100)

data_x = []
x = 0
for i in range(0, len(normalized_list)):
    data_x.append(x)
    x += 0.1

list_of_lists = []
list_of_lists.append(['church organ'] + list(zip(data_x, normalized_list)))

sonify.create_midi_from_data(list_of_lists, track_type='multiple', key='c_major', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

As always, we insert the sonification to the corresponding video (in this case to the complete sequence of pictures):

In [ ]:
from IPython.display import HTML
from base64 import b64encode

!ffmpeg -y -loglevel quiet -i complete_fly_video.mp4 -i mag.wav -c:v copy -c:a aac mag.mp4

mp4 = open('mag.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The same video with high resolution on YouTube:

In [ ]:
import IPython

IPython.display.YouTubeVideo('nXT9t3kGm1o')

## Sonification 6: pictures

In the section **Sonifying picture means**, we sonify the means of the pixel values of the images. We are going to work a little more on this since it is the information whose sonification will better synchronize with the animation of said images.

In order not to make the video too long and so that it has a smooth beginning and end, we are going to work only with the data that goes from `atlantes_186.jpg` picture to `atlantes_431.jpg`.

We begin by sonifying the means of the four data channels associated with the images for the range of pictures that we have just discussed:

In [ ]:
import os
import pandas
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE3)    # CSV input file complete path
snd_f = "1orbit_pictures_means"

data = pandas.read_csv(file, nrows=431, parse_dates=[0]).tail(246)        # Reading only rows between 186 and 431

# Columns in file: "datetime", "picture_file", "mean_global", "mean_r", "mean_g", "mean_b"
data_y1 = data.mean_global                    # Data in 'mean_global' column
data_y1_normalized = sonify.scale_list_to_range(data_y1, new_min=20, new_max=100)
data_y2 = data.mean_r                         # Data in 'mean_r' column
data_y2_normalized = sonify.scale_list_to_range(data_y2, new_min=20, new_max=100)
data_y3 = data.mean_g                         # Data in 'mean_g' column
data_y3_normalized = sonify.scale_list_to_range(data_y3, new_min=20, new_max=100)
data_y4 = data.mean_b                         # Data in 'mean_b' column
data_y4_normalized = sonify.scale_list_to_range(data_y4, new_min=20, new_max=100)

data_x = []
x = 0
for i in range(0, len(data_y1_normalized)):
    data_x.append(x)
    x += 0.1

multitrack_data = []
multitrack_data.append(list(zip(data_x, data_y1_normalized)))
multitrack_data.append(list(zip(data_x, data_y2_normalized)))
multitrack_data.append(list(zip(data_x, data_y3_normalized)))
multitrack_data.append(list(zip(data_x, data_y4_normalized)))

# Let's add some instruments to each track
instruments_to_add = ['steel drums', 'rock organ', 'pizzicato strings', 'oboe']

multitrack_data_with_instruments = []
for index, track in enumerate(multitrack_data):
    multitrack_data_with_instruments.append([instruments_to_add[index]] + track)

sonify.create_midi_from_data(multitrack_data_with_instruments, track_type='multiple', key='c_major', file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

Now we build the video with those images and we attach the previous sonification to it. As we did in the **Sonification 1**, before we start we have to prepare the photos by copying the ones we have selected (from `atlantes_186.jpg` to `atlantes_431.jpg`) and renaming them so that they have a correlative number from 001 to 246. We have made this selection on the `atlantes_1orbit` directory. To make the video, it only remains to execute the following cell:

In [ ]:
from IPython.display import HTML
from base64 import b64encode

!ffmpeg -y -loglevel quiet -f image2 -r 10 -i atlantes_1orbit/%03d.jpg -vf scale=320:240 -r:v 10 -c:v libx264 -qp 0 -preset veryslow -an 1orbit_video.mp4
!ffmpeg -y -loglevel quiet -i 1orbit_video.mp4 -i 1orbit_pictures_means.wav -c:v copy -c:a aac 1orbit_pictures_means.mp4

mp4 = open('1orbit_pictures_means.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The same video with high resolution on YouTube:

In [ ]:
import IPython

IPython.display.YouTubeVideo('SL1XojuapDE')

## Sonification 7: latitude

Another sonification that is surely interesting is that of the latitude of the station. This magnitude is sinusoidal in shape, which should produce a gradual, continuous sound variation. Although when watching the video the relationship between the sound and the images is not appreciated, the sonification helps to interpret what we are seeing by allowing the immediate identification of the latitude of the pictures.

In [ ]:
import os
import pandas
import numpy
import sonify
import IPython

file = os.path.join(path_atlantes, RESULTS_FILE)    # CSV ISS file complete path
snd_f = "latitude"

data = pandas.read_csv(file, parse_dates=[0])       # Reading whole file

# Columnas en fichero: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
data_x = data.datetime                              # Data in 'latitude' datetime
data_y = data.latitude                              # Data in 'latitude' column

fig, ax = plt.subplots()
ax.plot(data_x, data_y, label="Latitude")

plt.legend()
plt.grid()
plt.show()

In [ ]:
import os
import pandas
import sonify
import IPython
 
file = os.path.join(path_atlantes, RESULTS_FILE)     # CSV ISS file complete path
snd_f = "latitude"

data = pandas.read_csv(file, parse_dates=[0])        # Reading whole file

# Columns in file: datetime,picture_file,latitude,longitude,temp_cpu,temp_h,temp_p,humidity,pressure,pitch,roll,yaw,mag_x,mag_y,mag_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
data_y1 = data.latitude                              # Data in 'latitude' column
data_y1_normalized = sonify.scale_list_to_range(data_y1, new_min=20, new_max=100)

final_data = []
x = 0
for i in range(0, len(data_y1_normalized)):
    final_data.append((x, data_y1_normalized[i]))
    x += 0.1

sonify.create_midi_from_data(final_data, file=snd_f+".mid")

# Converting the MIDI file into WAV to play it in the notebook
!fluidsynth /content/default.sf2 -F {snd_f}.wav -i -n -T wav {snd_f}.mid
IPython.display.Audio(snd_f+".wav")

Creamos el vídeo stopmotion con esta sonificación:

In [ ]:
from IPython.display import HTML
from base64 import b64encode

#!ffmpeg -y -loglevel quiet -f image2 -r 10 -i atlantes/atlantes_%03d.jpg -vf scale=320:240 -r:v 10 -c:v libx264 -qp 0 -preset veryslow -an complete_fly_video.mp4
!ffmpeg -y -loglevel quiet -i complete_fly_video.mp4 -i latitude.wav -c:v copy -c:a aac latitude.mp4

mp4 = open('latitude.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

El mismo vídeo en alta resolución en YouTube:

In [ ]:
import IPython

IPython.display.YouTubeVideo('G8YluWG5hmw')



---


**THE END**